# Marvel Character and Comic Explorer

This Dash application allows users to explore Marvel Comics characters, their comic appearances, and connections to major events. It uses the Marvel Comics API and a static CSV for redundancy. The app is designed to be deployment-ready with a polished interface and explanatory visualizations.

## Requirements
- Install: `pip install dash plotly pandas requests dash-bootstrap-components python-dotenv beautifulsoup4 fuzzywuzzy wordcloud`
- Register at [Marvel Developer Portal](https://developer.marvel.com/) for API keys.
- Store keys in `.env`:
> - MARVEL_PUBLIC_KEY=your_public_key
> - MARVEL_PRIVATE_KEY=your_private_key

## Data
- **Source**: Marvel Comics API (characters, comics, events).
- **Static Backup**: `data/marvel_data.csv` (included in repo). Use this to avoid ~1-hour API scraping (see Data Loading Note below).
- **Character List**: Scraped from Comic Vine’s “Top 300 Most Published Marvel Characters”.

## Features
- **Components**: Dropdown (characters), Checklist (comic format), Date Picker Range (publication dates), Text Input (search comics/events).
- **Visualizations**: Bar chart (appearances), Line chart (trends), Network graph (character-event connections).
- **Interactivity**: Updates via a single callback.
- **Narrative**: Guides users through exploring Marvel's universe.

## Limitations
- **Character Aliases**: Some characters have multiple aliases (e.g., Spider-Man, Peter Parker, Miles Morales), which are not aggregated. For example, "Spider-Man" appears in ~400 comics in the CSV, but has many more under different names. This is due to the complexity of mapping aliases in the Marvel API and Comic Vine data.
- **API Constraints**: The Marvel API has a 3,000 calls/day limit.

## Instructions for Running
1. Save as `marvel_dash_app.ipynb`.
2. Place `data/marvel_data.csv` in `data/` or generate via Cell 4.
3. Install: `pip install -r requirements.txt`.
4. Set up `.env` with API keys.
5. Run all cells. Access at `http://127.0.0.1:8050`.

**Troubleshooting**: Check dependencies and API keys if the app fails to load.

In [ ]:
# import libraries and set up environment
import os
import requests
import hashlib
import time
import pandas as pd
import re
import networkx as nx
from datetime import datetime
from dotenv import load_dotenv
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from fuzzywuzzy import process
import unicodedata
from wordcloud import WordCloud
import base64
from io import BytesIO
from bs4 import BeautifulSoup 

# load environment variables
load_dotenv()
public_key = os.getenv('MARVEL_PUBLIC_KEY', 'your_public_key')
private_key = os.getenv('MARVEL_PRIVATE_KEY', 'your_private_key')


In [ ]:
def normalize_for_fuzzy(s):
    """
    Normalize string for fuzzy matching by removing non-alphanumeric characters,
    converting to lowercase, and normalizing unicode characters.
    
    Args:
        s (str): input string to normalize.
    Returns:
        str: normalized string suitable for fuzzy matching.
    """
    
    s = s.lower()
    s = re.sub(r'\W+', '', s) 
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')
    return s

def standardize_name(name):
    """
    Standardizes character names by removing unnecessary content, correcting common
    formatting issues, and applying consistent capitalization.

    Args:
        name (str): character name to standardize.
    Returns:
        str: standardized character name, or None if input is invalid.
    """

    if not name or pd.isna(name):
        return None

    # remove escaped quotes and parentheses content
    name = re.sub(r'\\"', '', name)
    name = re.sub(r'\(.*?\)', '', name)

    # remove extra spaces and strip
    name = re.sub(r'\s+', ' ', name).strip().lower()

    # manual replacements for known cases
    replacements = {
        'spiderman': 'Spider-Man',
        'ironman': 'Iron Man',
        'captainamerica': 'Captain America',
        'dr. strange': 'Doctor Strange',
        'dr strange': 'Doctor Strange',
        'antman': 'Ant-Man',
        'blackwidow': 'Black Widow',
        'wolverine': 'Wolverine',
        'ms. marvel': 'Ms. Marvel',
        'nick fury': 'Nick Fury',
        'jean grey': 'Jean Grey',
        'hawkeye': 'Hawkeye',
        'captain marvel': 'Captain Marvel',
        'scarlet witch': 'Scarlet Witch',
        'deadpool': 'Deadpool',
    }
    if name in replacements:
        return replacements[name]

    # capitalize hyphenated names correctly
    if '-' in name:
        name = '-'.join(part.capitalize() for part in name.split('-'))
    else:
        name = ' '.join(part.capitalize() for part in name.split())

    return name

def scrape_comicvine_characters():
    """
    Scrapes top 300 most published Marvel characters from ComicVine.
    Returns:
        list: list of standardized character names.
    """
    # set up base URL and prepare to scrape
    base_url = "https://comicvine.gamespot.com/profile/moonofcomics/lists/top-300-most-published-marvel-characters/80119/"
    all_character_names = []

    for page in range(1, 5):
        url = base_url if page == 1 else f"{base_url}?page={page}"
        print(f"Scraping page {page} ...")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # find all <h3> tags, filter and clean names
        h3_tags = soup.find_all("h3")
        for h3 in h3_tags:
            name = h3.get_text(strip=True)
            if name.lower() != "stan lee":  # skip "Stan Lee" honorable mention
                std_name = standardize_name(name)
                if std_name:
                    all_character_names.append(std_name)

    # deduplicate and sort
    unique_names = sorted(set(all_character_names))
    return unique_names

def main():
    """
    Main function to scrape character names and save them to a CSV file.
    """
    
    names = scrape_comicvine_characters()
    print(f"Total unique characters scraped: {len(names)}")

    df = pd.DataFrame({'standardized_name': names})
    df.to_csv("top_marvel_characters.csv", index=False)
    print("✅ Saved standardized character names to top_marvel_characters.csv")

if __name__ == "__main__":
    main()

In [ ]:
def fetch_all_comics_for_character(char_id, base_url, public_key, private_key):
    """
    Fetch all comics for a character from Marvel API.

    Args:
        char_id (int): character ID.
        base_url (str): API base URL.
        public_key (str): public API key.
        private_key (str): private API key.

    Returns:
        list: list of comic data dictionaries.
    """
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    
    all_comics = []
    limit = 50
    offset = 0

    while True:
        ts = str(int(time.time()))
        hash_val = hashlib.md5((ts + private_key + public_key).encode()).hexdigest()

        comics_url = (
            f"{base_url}characters/{char_id}/comics"
            f"?ts={ts}&apikey={public_key}&hash={hash_val}&limit={limit}&offset={offset}"
        )

        try:
            response = session.get(comics_url, timeout=10)  # set explicit timeout
            response.raise_for_status()
            data = response.json().get('data', {})
        except requests.exceptions.Timeout:
            print(f"  Timeout fetching comics at offset {offset} for character ID {char_id}. Skipping this batch.")
            offset += limit  # move to next batch
            time.sleep(1)  # longer delay before retrying
            continue
        except requests.exceptions.RequestException as e:
            print(f"  Error fetching comics at offset {offset} for character ID {char_id}: {e}")
            break  # stop on non-timeout errors

        comics = data.get('results', [])
        all_comics.extend(comics)
        print(f"  Fetched {len(comics)} comics at offset {offset} (total: {len(all_comics)})")

        total = data.get('total', 0)
        count = data.get('count', 0)
        offset += count

        if offset >= total or count == 0:
            break

        time.sleep(0.5)

    return all_comics

In [ ]:
def fetch_marvel_data_from_list(filename='top_marvel_characters.csv'):
    """
    Fetch comic data for a list of Marvel characters from Marvel API and save to CSV.

    Args:
        filename (str): path to CSV file containing character names (default: 'top_marvel_characters.csv').

    Returns:
        pandas.DataFrame: df containing fetched comic data.
    """
    if not public_key or not private_key:
        raise ValueError("Invalid API keys. Please set MARVEL_PUBLIC_KEY and MARVEL_PRIVATE_KEY in .env.")

    df_names = pd.read_csv(filename)
    character_names = df_names['standardized_name'].dropna().unique().tolist()

    base_url = 'https://gateway.marvel.com/v1/public/'
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    data = []

    for name in character_names:
        ts = str(int(time.time()))
        hash_val = hashlib.md5((ts + private_key + public_key).encode()).hexdigest()
        search_url = f"{base_url}characters?ts={ts}&apikey={public_key}&hash={hash_val}&nameStartsWith={name}"

        print(f"Searching for {name}...")
        try:
            response = session.get(search_url, timeout=10)
            response.raise_for_status()
            results = response.json().get('data', {}).get('results', [])
        except requests.exceptions.Timeout:
            print(f"  Timeout searching for {name}. Skipping.")
            continue
        except requests.exceptions.RequestException as e:
            print(f"  Error fetching data for {name}: {e}")
            continue

        if not results:
            print(f"  No matches found for {name}")
            continue

        # fuzzy matching to find best match
        all_names = [char['name'] for char in results]
        best_match, score = process.extractOne(name, all_names)

        if score < 70:
            print(f"  Low confidence match for {name}: {best_match} (score: {score})")
            continue

        matched_char = next((char for char in results if char['name'] == best_match), None)
        if not matched_char:
            continue

        char_id = matched_char['id']

        try:
            comics = fetch_all_comics_for_character(char_id, base_url, public_key, private_key)
            print(f"  Successfully fetched {len(comics)} comics for character {best_match}")
        except Exception as e:
            print(f"  Error fetching comics for {best_match}: {e}")
            continue

        for comic in comics:
            dates = comic.get('dates', [])
            on_sale_date = next((d['date'] for d in dates if d['type'] == 'onsaleDate'), '1900-01-01')
            events_items = comic.get('events', {}).get('items', [])
            event_name = events_items[0]['name'] if events_items else 'None'

            data.append({
                'character': best_match,
                'comic_title': comic['title'],
                'format': comic.get('format', 'Comic'),
                'date': pd.to_datetime(on_sale_date, errors='coerce'),
                'event': event_name
            })

        time.sleep(1)  # increased delay between characters

    df = pd.DataFrame(data)
    if not df.empty:
        os.makedirs('data', exist_ok=True)
        df.to_csv('data/marvel_data.csv', index=False)
        print(f"\n✅ Successfully created marvel_data.csv with {len(df)} records")
    else:
        print("⚠️ No data fetched. CSV not created.")
    return df

In [ ]:
def load_data():
    """
    Load Marvel data from CSV file if it exists, otherwise fetch from API.
    Returns:
        pandas.DataFrame: df containing Marvel comic data.
    """
    
    if os.path.exists('data/marvel_data.csv'):
        print("Loading data from static CSV: data/marvel_data.csv")
        df = pd.read_csv('data/marvel_data.csv', parse_dates=['date'])
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df.dropna(subset=['date'], inplace=True)
        return df
    else:
        print("Static CSV not found. Fetching data from Marvel API (this may take ~1 hour)...")
        return fetch_marvel_data_from_list()

## Data Loading Note

The next cell (`df = load_data()`) loads the Marvel Comics data. By default, it uses the static file `data/marvel_data.csv` if available. If the file is missing, it will fetch data from the Marvel API, which can take **approximately 1 hour** due to the large number of characters and API rate limits (3,000 calls/day).

**Options**:
- **Use the Static CSV**: Ensure `data/marvel_data.csv` is in the `data/` folder (included in the repository). This is the fastest way to run the app without scraping.
- **Fetch Fresh Data**: If you prefer to scrape the latest data, ensure valid API keys are set in `.env`. Be prepared for a long runtime and potential API limit issues.

**Recommendation**: Use the provided `data/marvel_data.csv` unless you specifically need fresh data. To generate a new CSV, run the cell below, but allow sufficient time.

In [ ]:
df = load_data()

In [ ]:
# initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://fonts.googleapis.com/css2?family=Bangers&display=swap'])

# define layout
app.layout = dbc.Container([
    # marvel logo
    html.Img(
        src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Marvel_Logo.svg/960px-Marvel_Logo.svg.png?20161025051221',
        style={'width': '200px', 'display': 'block', 'margin': 'auto'}
    ),
    # title
    html.H1('Marvel Comics Universe Explorer', className='text-center my-4', style={'fontFamily': 'Bangers, cursive', 'color': '#E6242A'}),
    # introductory text
    html.P(
        'Dive into the Marvel Comics universe! Select a character to explore their comic appearances, filter by format or date range, '
        'and search for specific comics or events (e.g., "Civil War"). '
        'The charts below show appearance counts, publication trends, and character-event connections.',
        className='text-center mb-4', style={'fontSize': '16px'}
    ),

    # summary card
    dbc.Card([
        dbc.CardBody([
            html.H4("Character Summary", className="card-title"),
            html.P(id='summary-text', className="card-text")
        ])
    ], style={'marginBottom': '20px', 'backgroundColor': '#F8F9FA'}),

    # input components
    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                dbc.Col([html.Label('Select Character', style={'color': '#E6242A'}), dcc.Dropdown(id='character-dropdown')], width=4),
                dbc.Col([html.Label('Comic Format', style={'color': '#E6242A'}), dcc.Checklist(id='format-checklist', inline=True)], width=4),
                dbc.Col([html.Label('Search Comic/Event', style={'color': '#E6242A'}), dcc.Input(id='search-input', type='text', placeholder='e.g., Civil War')], width=4)
            ], className='mb-3'),
            dbc.Row([
                dbc.Col([html.Label('Publication Date Range', style={'color': '#E6242A'}), dcc.DatePickerRange(id='date-range')], width=12)
            ])
        ])
    ], style={'backgroundColor': '#F8F9FA', 'padding': '20px', 'marginBottom': '20px'}),

    # controls
    dbc.Row([
        dbc.Col([
            html.Label('Chart Style'),
            dcc.RadioItems(
                id='theme-toggle',
                options=[{'label': 'Light', 'value': 'light'}, {'label': 'Dark', 'value': 'dark'}],
                value='light',
                inline=True
            ),
            html.Button('Reset Filters', id='reset-button', n_clicks=0, className='btn btn-danger mt-2'),
            html.Button('Download Filtered Data', id='download-button', n_clicks=0, className='btn btn-primary mt-2 ml-2'),
            dcc.Download(id='download-data')
        ])
    ], className='my-3'),

    # graphs
    dbc.Row([
        dbc.Col(dcc.Graph(id='bar-chart'), width=6),
        dbc.Col(dcc.Graph(id='line-chart'), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='network-graph'), width=12)
    ]),
    dbc.Row([
        dbc.Col(html.Img(id='wordcloud'), width=12, className='text-center')
    ]),
    html.Div(id='peak-year', className='text-center my-2 font-weight-bold'),

    # captions
    html.P('Left: Comic appearances for selected character. Right: Publication trends over time.', className='text-center'),
    html.P('Character-event connections across the Marvel universe.', className='text-center'),
    html.P('Word cloud of comic titles for the selected filters.', className='text-center')

], fluid=True)

# callback for reset button
@app.callback(
    [Output('character-dropdown', 'options'),
     Output('character-dropdown', 'value'),
     Output('format-checklist', 'options'),
     Output('format-checklist', 'value'),
     Output('date-range', 'min_date_allowed'),
     Output('date-range', 'max_date_allowed'),
     Output('date-range', 'start_date'),
     Output('date-range', 'end_date'),
     Output('search-input', 'value')],
    [Input('reset-button', 'n_clicks')]
)

# callback to initialize or reset dropdowns and date range
def initialize_or_reset(n_clicks):
    """
    Initialize or reset dropdowns and date range inputs.
    Args:
        n_clicks (int): number of times reset button has been clicked.
    Returns:
        tuple: contains options for character dropdown, selected character, format checklist options,
               selected formats, min and max dates for date range, start and end dates, and search input value.
    """
    characters = [{'label': c, 'value': c} for c in df['character'].dropna().unique()]
    formats_unique = df['format'].dropna().unique()
    formats = [{'label': f, 'value': f} for f in formats_unique]
    min_date = df['date'].min()
    max_date = df['date'].max()

    return characters, df['character'].dropna().unique()[0], formats, list(formats_unique), min_date, max_date, min_date, max_date, ''

# generate word cloud
def generate_wordcloud(text, theme):
    """
    Generate a word cloud image from a list of comic titles.
    Args:
        text (list): list of comic titles.
        theme (str): theme for background color ('light' or 'dark').
    Returns:
        str: base64 encoded image source for word cloud.
    """
    bg_color = 'white' if theme == 'light' else '#333333'
    wc = WordCloud(width=600, height=400, background_color=bg_color).generate(' '.join(text))
    buf = BytesIO()
    wc.to_image().save(buf, format='PNG')
    data = base64.b64encode(buf.getvalue()).decode()
    return f'data:image/png;base64,{data}'

# main callback for updating plots and summary
@app.callback(
    [Output('bar-chart', 'figure'),
     Output('line-chart', 'figure'),
     Output('network-graph', 'figure'),
     Output('wordcloud', 'src'),
     Output('peak-year', 'children'),
     Output('summary-text', 'children')],
    [Input('character-dropdown', 'value'),
     Input('format-checklist', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('search-input', 'value'),
     Input('theme-toggle', 'value')]
)

def update_plots(character, formats, start_date, end_date, search, theme):
    """
    Update bar chart, line chart, network graph, word cloud, peak year message, and summary card based on user inputs.
    Args:
        character (str): selected character name.
        formats (list): selected comic formats.
        start_date (str): start date for filtering comics.
        end_date (str): end date for filtering comics.
        search (str): search term for comics or events.
        theme (str): theme for plots ('light' or 'dark').
    Returns:
        tuple: updated bar chart, line chart, network graph, word cloud source, peak year message, and summary text.
    """
    if character is None:
        character = df['character'].unique()[0]
    if formats is None:
        formats = df['format'].unique().tolist()
    if start_date is None:
        start_date = df['date'].min()
    if end_date is None:
        end_date = df['date'].max()

    template = 'plotly_dark' if theme == 'dark' else 'plotly_white'

    filtered_df = df[
        (df['character'] == character) &
        (df['format'].isin(formats)) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]

    if search:
        filtered_df = filtered_df[
            filtered_df['comic_title'].str.contains(search, case=False, na=False) |
            filtered_df['event'].str.contains(search, case=False, na=False)
        ]

    if filtered_df.empty:
        bar_fig = px.bar(title='No data available for the selected filters.')
        bar_fig.update_layout(
            xaxis_title='Character',
            yaxis_title='Number of Appearances',
            font=dict(family='Arial', size=12),
            template=template
        )
        line_fig = px.line(title='No data available for the selected filters.')
        line_fig.update_layout(
            xaxis_title='Publication Date',
            yaxis_title='Number of Comics',
            font=dict(family='Arial', size=12),
            template=template
        )
        network_fig = go.Figure().add_annotation(
            text='No data available.',
            showarrow=False,
            xref='paper', yref='paper',
            x=0.5, y=0.5,
            font=dict(size=20)
        )
        network_fig.update_layout(
            title='Character-Event Connections',
            showlegend=False,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            font=dict(family='Arial', size=12),
            template=template
        )
        wordcloud_src = ''
        peak_msg = 'No data to determine peak year.'
        summary_text = 'No data available.'
    else:
        # bar chart: comic appearances
        bar_data = filtered_df.groupby('character').size().reset_index(name='appearances')
        bar_fig = px.bar(
            bar_data,
            x='character',
            y='appearances',
            title=f'Comic Appearances for {character}',
            color_discrete_sequence=['#E6242A']
        )
        bar_fig.update_layout(
            xaxis_title='Character',
            yaxis_title='Number of Appearances',
            font=dict(family='Arial', size=12),
            template=template
        )

        # line chart: yearly comic releases
        line_data = filtered_df.groupby(filtered_df['date'].dt.year).size().reset_index(name='comic_count')
        line_fig = px.line(
            line_data,
            x='date',
            y='comic_count',
            title=f'Yearly Comic Releases for {character}',
            markers=True,
            color_discrete_sequence=['#FFD700']
        )
        line_fig.update_layout(
            xaxis_title='Year',
            yaxis_title='Number of Comics',
            font=dict(family='Arial', size=12),
            template=template,
            yaxis_type='linear'
        )

        # network graph: character-event connections
        network_data = filtered_df[filtered_df['event'] != 'None'][['character', 'event']].drop_duplicates()
        nodes = list(set(network_data['character']).union(set(network_data['event'])))
        node_dict = {n: i for i, n in enumerate(nodes)}
        edges = [(node_dict[row['character']], node_dict[row['event']]) for _, row in network_data.iterrows()]

        G = nx.Graph()
        G.add_edges_from([(row['character'], row['event']) for _, row in network_data.iterrows()])
        pos = nx.kamada_kawai_layout(G)
        x_nodes = [pos[node][0] for node in nodes]
        y_nodes = [pos[node][1] for node in nodes]
        network_fig = go.Figure()
        for edge in edges:
            x0, y0 = x_nodes[edge[0]], y_nodes[edge[0]]
            x1, y1 = x_nodes[edge[1]], y_nodes[edge[1]]
            network_fig.add_trace(go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(color='gray', width=1),
                hoverinfo='none'
            ))
        network_fig.add_trace(go.Scatter(
            x=x_nodes,
            y=y_nodes,
            mode='markers+text',
            text=nodes,
            marker=dict(size=20, color=['#E6242A' if 'event' not in str(n) else '#FFD700' for n in nodes]),
            textposition='top center',
            hovertemplate='%{text}<br>Type: %{customdata}<extra></extra>',
            customdata=['Character' if 'event' not in str(n) else 'Event' for n in nodes]
        ))
        network_fig.update_layout(
            title='Character-Event Connections',
            showlegend=False,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            font=dict(family='Arial', size=12),
            margin=dict(l=20, r=20, t=40, b=20),
            template=template
        )

        # word cloud
        wordcloud_src = generate_wordcloud(filtered_df['comic_title'].dropna().tolist(), theme) if not filtered_df.empty else ''

        # peak year message
        peak_msg = f"Peak comic activity: {filtered_df['date'].dt.year.value_counts().idxmax()}"

        # summary text
        total_appearances = len(filtered_df)
        unique_events = filtered_df['event'].nunique()
        top_format = filtered_df['format'].mode()[0] if not filtered_df['format'].empty else 'N/A'
        summary_text = (
            f"Total Comics: {total_appearances} | "
            f"Unique Events: {unique_events} | "
            f"Top Format: {top_format}"
        )

    return bar_fig, line_fig, network_fig, wordcloud_src, peak_msg, summary_text

# callback for download button
@app.callback(
    Output('download-data', 'data'),
    Input('download-button', 'n_clicks'),
    State('character-dropdown', 'value'),
    State('format-checklist', 'value'),
    State('date-range', 'start_date'),
    State('date-range', 'end_date'),
    State('search-input', 'value'),
    prevent_initial_call=True
)
def download_data(n_clicks, character, formats, start_date, end_date, search):
    """
    Generate a CSV file of filtered dataset for download.
    Args:
        n_clicks (int): number of times download button was clicked.
        character (str): selected character.
        formats (list): selected formats.
        start_date (str): start date.
        end_date (str): end date.
        search (str): search term.
    Returns:
        dict: data for dcc.Download to trigger CSV download.
    """
    filtered_df = df[
        (df['character'] == character) &
        (df['format'].isin(formats)) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]
    if search and search.strip():
        filtered_df = filtered_df[
            filtered_df['comic_title'].str.contains(search, case=False, na=False) |
            filtered_df['event'].str.contains(search, case=False, na=False)
        ]
    return dcc.send_data_frame(filtered_df.to_csv, f"marvel_comics_{character}.csv")

# run app
if __name__ == '__main__':
    app.run(debug=True, port=8050)